## SVM Model

The code here represents the SVM model to predict whether a person is affected by covid or pneumonia or whether the person is in normal state using the chest X-Ray images given as dataset files. The Chest X-Ray images have been preproccessed and then the features have been extracted and evaluated and stored in .mat files. We now load these files and create the SVM model.

In [43]:
# Importing the required packages from the necessary libraries
from utils import *

import os
import pywt
import cv2
import glob
import itertools
import numpy as np
import seaborn as sn
import pandas as pd
import scipy.io as sio
import skimage.io as io
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.svm import SVC
from tensorflow import keras
from skimage import exposure
from scipy.stats import skew
from scipy.stats import kurtosis
from sklearn import preprocessing
from IPython.display import Image
from skimage.color import rgb2gray
from keras.utils import to_categorical
from skimage.feature import greycomatrix
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import KernelPCA
from tensorflow.keras.utils import plot_model
from skimage.transform import  rescale,resize
from sklearn.preprocessing import MinMaxScaler
from skimage.util import img_as_uint,img_as_ubyte
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [45]:
# Mentioning the working directory
source_dir='./'

In [46]:
# Accessing covid.mat file and getting the data from the file

covid_features=sio.loadmat(os.path.join(source_dir,'covid.mat')) 
covid_features=covid_features['covid'] 

# Accessing normal.mat file and getting the data from the file
normal_features=sio.loadmat(os.path.join(source_dir,'normal.mat')) 
normal_features=normal_features['normal']  

# Accessing pneumonia.mat file and getting the data from the file
pneumonia_features=sio.loadmat(os.path.join(source_dir,'pneumonia.mat')) 
pneumonia_features=pneumonia_features['pneumonia']  

In [47]:
# Extracting the scores-i.e the inputs and storing it in X
X=np.concatenate((covid_features[:,:-1],normal_features[:,:-1],pneumonia_features[:,:-1]), axis=0)#inputs

# Extracting the target labels, the last column alone
y=np.concatenate((covid_features[:,-1],normal_features[:,-1],pneumonia_features[:,-1]), axis=0)#target labels

In [48]:
# Normalization of the data between 0 and 1
min_max_scaler=MinMaxScaler()
X = min_max_scaler.fit_transform(X)

In [49]:
# We use Kernel to reduce the feature components to 64 for the input data
transformer = KernelPCA(n_components=64, kernel='linear')
X = transformer.fit_transform(X)

In [50]:

m,n = np.shape(X)
# Weight matrix
W = cp.Variable(n)
# Slack variable
Psi = cp.Variable(m)
# Gamma term
b = cp.Variable()
# Control/Regularization paramters
C = 0.01

In [51]:
# Soft-margin SVM
# Objective function for softmargin svm is 0.5W'W + Ce'Psi
# cvxpy command for computing the transpose of a matrix is cp.transpose(W)
objective = cp.Minimize(0.5 * cp.square(cp.norm(W)) + C * cp.sum(Psi))
# Constraint y(W'x+b)+Psi-1>=0; Psi>=0
constraints = [cp.matmul(y, X @ W + b)-1 + Psi >= 0, Psi >= 0]

prob = cp.Problem(objective, constraints)
prob.solve()
print("Status of optimization:", prob.status)

print("Optimal value of W:", W.value)
print("Optimal value of b:", b.value)

print("Constraint_0:", constraints[0])
print("Constraint_1:", constraints[1]) # second constraint is available for soft-margin svm

print("Dual value for the first constraint:", constraints[0].dual_value) # Value of the lagrangian multiplier
print("Dual value for the second constraint:", constraints[1].dual_value) # second constraint is available for soft-margin svm

Status of optimization: optimal_inaccurate
Optimal value of W: [ 6.09101266e-13 -8.57988505e-13 -5.21635725e-12 -2.34232047e-13
 -3.34947855e-12  3.98692813e-13 -2.40659095e-12  1.91021222e-12
 -4.92378021e-13 -3.94436836e-13  3.05675508e-14  1.44368178e-13
 -4.89169900e-13 -1.12934434e-13  4.78766675e-14 -3.02147149e-13
  4.86806399e-14  1.20227201e-13  1.38241800e-13  3.94192733e-14
 -1.14012148e-13  4.52704219e-14  7.36892985e-15 -9.45934653e-14
  4.95572858e-14  3.08921135e-14 -3.26791055e-15  6.88786796e-14
 -1.36404361e-13 -4.42373146e-15  6.85576285e-15  3.33338084e-14
  5.26451094e-14 -4.00245711e-15 -4.19240690e-14 -5.81282697e-14
  2.11846869e-14  8.67407458e-14 -1.58580001e-13 -6.34866315e-14
 -6.16271965e-14  5.62102363e-14 -8.12224087e-14 -4.35317243e-14
  1.36187773e-14 -1.15749814e-14  8.59667787e-15  3.67832468e-14
  1.35866121e-14 -5.09135290e-14  7.29046442e-15 -2.16928355e-14
 -4.42077883e-14  1.02355279e-14 -7.93775304e-15  6.19998130e-15
  7.30566156e-14  3.6710077

C:\Users\iamta\Anaconda3\lib\site-packages\cvxpy\problems\problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
